# Volvo Trucks Analytics

Imports and global variables go here

In [108]:
import pandas as pd
import numpy as np
import csv
truck1csv = "../data/trucks/truck1.csv"
truck2csv = "../data/trucks/truck2.csv"
rowNaNThresholdPercent = 75

Define all cleaning methods here

In [106]:
def readCsv (truck):
    return pd.read_csv(truck, header=[0])
    
def cullUtcCols (truckData):
    noUTC = truckData.drop(["UTC hour", "UTC minute", "UTC second", "UTC month", "UTC day", "UTC year"], axis = 1)
    return noUTC

def removeUnnecessaryRows(dataFrame): 
    '''Finds threshold of Non-NA type using percentage and deletes rows.'''
    lenght = len(dataFrame.columns)
    TValue = (rowNaNThresholdPercent/100) * lenght
    print("Threshold value : " + str(int(TValue)))
    return dataFrame.dropna(thresh=int(TValue))

Cleaning of Truck 1 Data starts here

In [107]:
truck1data = readCsv(truck1csv)
truck1data = cullUtcCols(truck1data)
truck1data = removeUnnecessaryRows(truck1data)

# After cleaning, check the shape of the dataframe
truck1data.shape

Threshold value : 33


(13, 45)

Cleaning of Truck 2 Data starts here

In [ ]:
truck2data = readCsv(truck2csv)
truck2data = cullUtcCols(truck2data)
truck2data = removeUnnecessaryRows(truck2data)

# After cleaning, check the shape of the dataframe
truck2data.shape